In [2]:
import torch
from torch import optim, nn
from torch.utils.data import Dataset, DataLoader
import pandas as pd

In [3]:
class CustomDataset(Dataset):
    def __init__(self, file_path):
        df = pd.read_csv(file_path)
        self.x = df.iloc[:, 0].values 
        self.y = df.iloc[:, 1].values 
        self.length = len(df)
        
    def __getitem__(self, index):
        x = torch.FloatTensor([self.x[index] ** 2, self.x[index]])
        y = torch.FloatTensor([self.y[index]])
        return x, y
    
    def __len__(self):
        return self.length

In [4]:
class CustomModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer = nn.Linear(2, 1)
        
    def forward(self, x):
        x = self.layer(x)
        return x

In [5]:
train_dataset = CustomDataset("non_linear.csv")
train_dataloader = DataLoader(train_dataset, batch_size=128, shuffle=True, drop_last=True)

In [6]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model = CustomModel().to(device)
criterion = nn.MSELoss().to(device)
optimizer = optim.SGD(model.parameters(), lr=1e-4)

In [7]:
for epoch in range(10000):
    cost = 0.0
    
    for x, y in train_dataloader:
        x = x.to(device)
        y = y.to(device)
        
        output = model(x)
        loss = criterion(output, y)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        cost += loss
        
    cost = cost / len(train_dataloader)
    
    if (epoch + 1) % 1000 == 0:
        print(f"Epcoh : {epoch + 1}, Model : {list(model.parameters())}, Cost : {cost:.3f}")

Epcoh : 1000, Model : [Parameter containing:
tensor([[ 3.1065, -1.7004]], requires_grad=True), Parameter containing:
tensor([0.0957], requires_grad=True)], Cost : 0.163
Epcoh : 2000, Model : [Parameter containing:
tensor([[ 3.1062, -1.7031]], requires_grad=True), Parameter containing:
tensor([0.1296], requires_grad=True)], Cost : 0.142
Epcoh : 3000, Model : [Parameter containing:
tensor([[ 3.1053, -1.7029]], requires_grad=True), Parameter containing:
tensor([0.1606], requires_grad=True)], Cost : 0.129
Epcoh : 4000, Model : [Parameter containing:
tensor([[ 3.1052, -1.7032]], requires_grad=True), Parameter containing:
tensor([0.1890], requires_grad=True)], Cost : 0.126
Epcoh : 5000, Model : [Parameter containing:
tensor([[ 3.1043, -1.7032]], requires_grad=True), Parameter containing:
tensor([0.2148], requires_grad=True)], Cost : 0.094
Epcoh : 6000, Model : [Parameter containing:
tensor([[ 3.1040, -1.7028]], requires_grad=True), Parameter containing:
tensor([0.2386], requires_grad=True)],

In [8]:
with torch.no_grad():
    model.eval()
    inputs = torch.FloatTensor(
        [
            [1 ** 2, 1],
            [5 ** 2, 5],
            [11 ** 2, 11]
        ]
    ).to(device)
    outputs = model(inputs)
    print(outputs)

tensor([[  1.7152],
        [ 69.3752],
        [357.0452]])


In [9]:
torch.save(
    model,
    "./models/model.pt"
)

torch.save(
    model.state_dict(),
    "./models/model_state_dict.pt"
)

In [10]:
import torch
from torch import optim, nn
from torch.utils.data import Dataset, DataLoader
import pandas as pd
from torch.utils.data import random_split

class CustomDataset(Dataset):
    def __init__(self, file_path):
        df = pd.read_csv(file_path)
        self.x = df.iloc[:, 0].values 
        self.y = df.iloc[:, 1].values 
        self.length = len(df)
        
    def __getitem__(self, index):
        x = torch.FloatTensor([self.x[index] ** 2, self.x[index]])
        y = torch.FloatTensor([self.y[index]])
        return x, y
    
    def __len__(self):
        return self.length
    
class CustomModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer = nn.Linear(2, 1)
        
    def forward(self, x):
        x = self.layer(x)
        return x

dataset = CustomDataset("non_linear.csv")
dataset_size = len(dataset)
train_size = int(dataset_size * 0.8)
validation_size = int(dataset_size * 0.1)
test_size = int(dataset_size * 0.1)

train_dataset, validation_dataset, test_dataset = random_split(dataset, [train_size, validation_size, test_size])
print(f"Trianing Data Size : {len(train_dataset)}")
print(f"Validation Data Size : {len(validation_dataset)}")
print(f"Testing Data Size : {len(test_dataset)}")

train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True, drop_last=True)
validation_dataloader = DataLoader(validation_dataset, batch_size=4, shuffle=True, drop_last=True)
test_dataloader = DataLoader(test_dataset, batch_size=4, shuffle=True, drop_last=True)

device = "cuda" if torch.cuda.is_available() else "cpu"
model = CustomModel().to(device)
criterion = nn.MSELoss().to(device)
optimizer = optim.SGD(model.parameters(), lr=1e-4)

for epoch in range(10000):
    cost = 0.0
    
    for batch in train_dataloader:
        x, y = batch 
        x, y = x.to(device), y.to(device)
        
        output = model(x)
        loss = criterion(output, y)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        cost += loss
    cost = cost / len(train_dataloader)
    
    if (epoch + 1) % 1000 == 0:
        print(f"Epoch : {epoch + 1}, Model : {list(model.parameters())}, Cost : {cost:.3f}")
        
with torch.no_grad(): # 필요없는 미분값 계산(Loss값 계산 과정)을 정지 -- 모델 평가 과정
    model.eval() # 모델을 평가 상태로 변환
    
    for batch in validation_dataloader:
        x, y = batch
        x, y = x.to(device), y.to(device)
        
        outputs = model(x)
        print(f"X : {x}")
        print(f"Y : {y}")
        print(f"Outputs : {outputs}")
        print("----------------------")

Trianing Data Size : 160
Validation Data Size : 20
Testing Data Size : 20
Epoch : 1000, Model : [Parameter containing:
tensor([[ 3.0993, -1.7037]], requires_grad=True), Parameter containing:
tensor([0.5857], requires_grad=True)], Cost : 0.078
Epoch : 2000, Model : [Parameter containing:
tensor([[ 3.0993, -1.7038]], requires_grad=True), Parameter containing:
tensor([0.5219], requires_grad=True)], Cost : 0.076
Epoch : 3000, Model : [Parameter containing:
tensor([[ 3.0996, -1.7039]], requires_grad=True), Parameter containing:
tensor([0.4959], requires_grad=True)], Cost : 0.076
Epoch : 4000, Model : [Parameter containing:
tensor([[ 3.1002, -1.7041]], requires_grad=True), Parameter containing:
tensor([0.4853], requires_grad=True)], Cost : 0.074
Epoch : 5000, Model : [Parameter containing:
tensor([[ 3.0990, -1.7041]], requires_grad=True), Parameter containing:
tensor([0.4809], requires_grad=True)], Cost : 0.075
Epoch : 6000, Model : [Parameter containing:
tensor([[ 3.0998, -1.7041]], require

<p>모델 저장 함수</p>
<code>
torch.save( 
    model,
    path
)
</code>

<p>모델 불러오기 함수</p>
<code>
model = torch.load(
    path,
    map_location  # 모델의 장치 상태(cpu, gpu)
)
</code>
<p style="color:red"> 단 위 코드에서 CustomModel 클래스를 만든것처럼 원 모델의 클래스를 만들어 놔야함 안만들면 AttributeError남 </p>

In [11]:
torch.save(model, 'test_model.pt')

In [14]:
import torch
from torch import nn

class CustomModel(nn.Module):
    pass

device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = torch.load("test_model.pt", map_location=device)
print(model)

CustomModel(
  (layer): Linear(in_features=2, out_features=1, bias=True)
)


C:\Users\dlwns\AppData\Local\Temp\ipykernel_6208\220081637.py:8: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load("test_model.pt", map_location=device)


<p>모델의 파라미터를 CustomModel에 똑같이 구현해야함</p>

In [15]:
torch.save(
    model.state_dict(),
    "model_state.pt"
) # 간단하게 모델의 학습 과정을 담지 않고 결과물인 파라미터만 저장하는 함수

In [17]:
import torch
from torch import nn

class CustomModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer = nn.Linear(2, 1)
        
    def forward(self, x):
        x = self.layer(x)
        return x
    
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = CustomModel().to(device)

model_state_dict = torch.load("model_state.pt", map_location=device)
model.load_state_dict(model_state_dict)

with torch.no_grad():
    model.eval()
    inputs = torch.FloatTensor(
        [
            [1 ** 2, 1],
            [5 ** 2, 5],
            [11 ** 2, 11]
        ]
    ).to(device)
    outputs = model(inputs)
    print(outputs)

tensor([[  1.8732],
        [ 69.4392],
        [356.7437]])


C:\Users\dlwns\AppData\Local\Temp\ipykernel_6208\2032743002.py:16: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_state_dict = torch.load("model_state.pt", map_location

In [18]:
# checkpoint 사용 (tensorflow callbacks 비슷한듯)

import torch
from torch import optim, nn
from torch.utils.data import Dataset, DataLoader
import pandas as pd
from torch.utils.data import random_split

class CustomDataset(Dataset):
    def __init__(self, file_path):
        df = pd.read_csv(file_path)
        self.x = df.iloc[:, 0].values 
        self.y = df.iloc[:, 1].values 
        self.length = len(df)
        
    def __getitem__(self, index):
        x = torch.FloatTensor([self.x[index] ** 2, self.x[index]])
        y = torch.FloatTensor([self.y[index]])
        return x, y
    
    def __len__(self):
        return self.length
    
class CustomModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer = nn.Linear(2, 1)
        
    def forward(self, x):
        x = self.layer(x)
        return x

dataset = CustomDataset("non_linear.csv")
dataset_size = len(dataset)
train_size = int(dataset_size * 0.8)
validation_size = int(dataset_size * 0.1)
test_size = int(dataset_size * 0.1)

train_dataset, validation_dataset, test_dataset = random_split(dataset, [train_size, validation_size, test_size])
print(f"Trianing Data Size : {len(train_dataset)}")
print(f"Validation Data Size : {len(validation_dataset)}")
print(f"Testing Data Size : {len(test_dataset)}")

train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True, drop_last=True)
validation_dataloader = DataLoader(validation_dataset, batch_size=4, shuffle=True, drop_last=True)
test_dataloader = DataLoader(test_dataset, batch_size=4, shuffle=True, drop_last=True)

device = "cuda" if torch.cuda.is_available() else "cpu"
model = CustomModel().to(device)
criterion = nn.MSELoss().to(device)
optimizer = optim.SGD(model.parameters(), lr=1e-4)
checkpoint = 1

for epoch in range(10000):
    cost = 0.0
    
    for batch in train_dataloader:
        x, y = batch 
        x, y = x.to(device), y.to(device)
        
        output = model(x)
        loss = criterion(output, y)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        cost += loss
    cost = cost / len(train_dataloader)
    
    if (epoch + 1) % 1000 == 0:
        torch.save(
            {
                "model": "CustomModel",
                "epoch": epoch, # 필수
                "model_state_dict": model.state_dict(), # 필수
                "optimizer_state_dict": optimizer.state_dict(), # 필수
                "cost": cost,
                "description": f"CustomModel 체크포인트-{checkpoint}"
            },
            f"models/checkpoint-{checkpoint}.pt",
        )
        checkpoint += 1
        print(f"Epoch : {epoch + 1}, Model : {list(model.parameters())}, Cost : {cost:.3f}")
        
with torch.no_grad(): # 필요없는 미분값 계산(Loss값 계산 과정)을 정지 -- 모델 평가 과정
    model.eval() # 모델을 평가 상태로 변환
    
    for batch in validation_dataloader:
        x, y = batch
        x, y = x.to(device), y.to(device)
        
        outputs = model(x)
        print(f"X : {x}")
        print(f"Y : {y}")
        print(f"Outputs : {outputs}")
        print("----------------------")

Trianing Data Size : 160
Validation Data Size : 20
Testing Data Size : 20
Epoch : 1000, Model : [Parameter containing:
tensor([[ 3.1069, -1.7018]], requires_grad=True), Parameter containing:
tensor([0.1193], requires_grad=True)], Cost : 0.129
Epoch : 2000, Model : [Parameter containing:
tensor([[ 3.1032, -1.7019]], requires_grad=True), Parameter containing:
tensor([0.3217], requires_grad=True)], Cost : 0.087
Epoch : 3000, Model : [Parameter containing:
tensor([[ 3.1000, -1.7021]], requires_grad=True), Parameter containing:
tensor([0.4035], requires_grad=True)], Cost : 0.079
Epoch : 4000, Model : [Parameter containing:
tensor([[ 3.0994, -1.7022]], requires_grad=True), Parameter containing:
tensor([0.4366], requires_grad=True)], Cost : 0.076
Epoch : 5000, Model : [Parameter containing:
tensor([[ 3.1001, -1.7023]], requires_grad=True), Parameter containing:
tensor([0.4501], requires_grad=True)], Cost : 0.077
Epoch : 6000, Model : [Parameter containing:
tensor([[ 3.1001, -1.7021]], require

In [2]:
# checkpoint 이어서 하는 코드

import torch
import pandas as pd
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader

class CustomDataset(Dataset):
    def __init__(self, file_path):
        df = pd.read_csv(file_path)
        self.x = df.iloc[:, 0].values 
        self.y = df.iloc[:, 1].values 
        self.length = len(df)
        
    def __getitem__(self, index):
        x = torch.FloatTensor([self.x[index] ** 2, self.x[index]])
        y = torch.FloatTensor([self.y[index]])
        return x, y
    
    def __len__(self):
        return self.length
    
class CustomModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer = nn.Linear(2, 1)
        
    def forward(self, x):
        x = self.layer(x)
        return x

dataset = CustomDataset("non_linear.csv")
dataset_size = len(dataset)
train_size = int(dataset_size * 0.8)
validation_size = int(dataset_size * 0.1)
test_size = int(dataset_size * 0.1)

train_dataset, validation_dataset, test_dataset = random_split(dataset, [train_size, validation_size, test_size])
print(f"Trianing Data Size : {len(train_dataset)}")
print(f"Validation Data Size : {len(validation_dataset)}")
print(f"Testing Data Size : {len(test_dataset)}")

train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True, drop_last=True)
validation_dataloader = DataLoader(validation_dataset, batch_size=4, shuffle=True, drop_last=True)
test_dataloader = DataLoader(test_dataset, batch_size=4, shuffle=True, drop_last=True)

device = "cuda" if torch.cuda.is_available() else "cpu"
model = CustomModel().to(device)
criterion = nn.MSELoss().to(device)
optimizer = optim.SGD(model.parameters(), lr=1e-4)

checkpoint = torch.load("models/checkpoint-6.pt")
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
checkpoint_epoch = checkpoint["epoch"]
checkpoint_description = checkpoint["description"]
print(checkpoint_description)

for epoch in range(checkpoint_epoch, 100000):
    cost = 0.0
    
    for batch in train_dataloader:
        x, y = batch 
        x, y = x.to(device), y.to(device)
        
        output = model(x)
        loss = criterion(output, y)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        cost += loss
        if (epoch + 1) % 1000 == 0:
            print(f"Epoch : {epoch+1:4d}, Model : {list(model.parameters())}, Cost : {cost:.3f}")

NameError: name 'random_split' is not defined